In [1]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

In [2]:
train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

/Users/ivan.bragin/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [4]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [5]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(train_dataset.all_objects):
#         o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
    

253000
254000
255000
256000
253000
254000
255000
256000


In [6]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(test_dataset.all_objects):
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min

0
1000
2000
0
1000
2000


In [7]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f and "P1_kk" not in f_name:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [8]:
columns = []
for p in ["",
          "P1_"
         ]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [9]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [10]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [11]:
Xtest1 = [getX(o) for o in test_dataset.all_objects]
Xtest1 = pd.concat(Xtest1, axis=1).T
Xtest1.columns=columns
Xtest1 = Xtest1.fillna(-999)

In [12]:
# X0 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 0])
# X1 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 1])
# Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]

In [13]:
# np.save("x0", X0)
# np.save("y0", Y0)
# np.save("x1", X1)
# np.save("y1", Y1)

In [14]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=42)

In [15]:
train_data = Pool(data=X_train,
                  cat_features=raw_cat_features,
                  label=np.log(Y_train))

val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
                  label=np.log(Y_val))

In [31]:
cat_model = CatBoostRegressor(
    learning_rate=0.01,
    iterations=8000,
    eval_metric="RMSE",
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)



0:	learn: 0.5822980	test: 0.6013412	best: 0.6013412 (0)	total: 12.9ms	remaining: 1m 43s
1:	learn: 0.5793127	test: 0.5982164	best: 0.5982164 (1)	total: 24.9ms	remaining: 1m 39s
2:	learn: 0.5763928	test: 0.5953127	best: 0.5953127 (2)	total: 34.9ms	remaining: 1m 33s
3:	learn: 0.5734273	test: 0.5923008	best: 0.5923008 (3)	total: 47.1ms	remaining: 1m 34s
4:	learn: 0.5704643	test: 0.5892499	best: 0.5892499 (4)	total: 55.7ms	remaining: 1m 29s
5:	learn: 0.5675641	test: 0.5861362	best: 0.5861362 (5)	total: 64.2ms	remaining: 1m 25s
6:	learn: 0.5647451	test: 0.5831826	best: 0.5831826 (6)	total: 78.2ms	remaining: 1m 29s
7:	learn: 0.5619165	test: 0.5803624	best: 0.5803624 (7)	total: 86.7ms	remaining: 1m 26s
8:	learn: 0.5592085	test: 0.5776396	best: 0.5776396 (8)	total: 95.7ms	remaining: 1m 24s
9:	learn: 0.5563802	test: 0.5748057	best: 0.5748057 (9)	total: 111ms	remaining: 1m 28s
10:	learn: 0.5536002	test: 0.5719057	best: 0.5719057 (10)	total: 119ms	remaining: 1m 26s
11:	learn: 0.5508567	test: 0.569

102:	learn: 0.4085701	test: 0.4175378	best: 0.4175378 (102)	total: 1.01s	remaining: 1m 17s
103:	learn: 0.4077734	test: 0.4166276	best: 0.4166276 (103)	total: 1.02s	remaining: 1m 17s
104:	learn: 0.4069009	test: 0.4157068	best: 0.4157068 (104)	total: 1.03s	remaining: 1m 17s
105:	learn: 0.4061129	test: 0.4148404	best: 0.4148404 (105)	total: 1.04s	remaining: 1m 17s
106:	learn: 0.4052865	test: 0.4138855	best: 0.4138855 (106)	total: 1.05s	remaining: 1m 17s
107:	learn: 0.4045868	test: 0.4131276	best: 0.4131276 (107)	total: 1.06s	remaining: 1m 17s
108:	learn: 0.4038003	test: 0.4121828	best: 0.4121828 (108)	total: 1.07s	remaining: 1m 17s
109:	learn: 0.4030574	test: 0.4112543	best: 0.4112543 (109)	total: 1.08s	remaining: 1m 17s
110:	learn: 0.4022698	test: 0.4104186	best: 0.4104186 (110)	total: 1.09s	remaining: 1m 17s
111:	learn: 0.4015384	test: 0.4095663	best: 0.4095663 (111)	total: 1.1s	remaining: 1m 17s
112:	learn: 0.4008413	test: 0.4088149	best: 0.4088149 (112)	total: 1.11s	remaining: 1m 17s


206:	learn: 0.3578914	test: 0.3591277	best: 0.3591277 (206)	total: 1.99s	remaining: 1m 14s
207:	learn: 0.3575585	test: 0.3587303	best: 0.3587303 (207)	total: 2s	remaining: 1m 14s
208:	learn: 0.3572973	test: 0.3585007	best: 0.3585007 (208)	total: 2.01s	remaining: 1m 14s
209:	learn: 0.3570657	test: 0.3582049	best: 0.3582049 (209)	total: 2.02s	remaining: 1m 14s
210:	learn: 0.3568361	test: 0.3579124	best: 0.3579124 (210)	total: 2.03s	remaining: 1m 14s
211:	learn: 0.3566258	test: 0.3576532	best: 0.3576532 (211)	total: 2.04s	remaining: 1m 14s
212:	learn: 0.3563209	test: 0.3572617	best: 0.3572617 (212)	total: 2.04s	remaining: 1m 14s
213:	learn: 0.3560305	test: 0.3569911	best: 0.3569911 (213)	total: 2.06s	remaining: 1m 14s
214:	learn: 0.3558635	test: 0.3568087	best: 0.3568087 (214)	total: 2.06s	remaining: 1m 14s
215:	learn: 0.3555925	test: 0.3564748	best: 0.3564748 (215)	total: 2.07s	remaining: 1m 14s
216:	learn: 0.3553193	test: 0.3561612	best: 0.3561612 (216)	total: 2.08s	remaining: 1m 14s
21

314:	learn: 0.3364907	test: 0.3353333	best: 0.3353333 (314)	total: 2.99s	remaining: 1m 13s
315:	learn: 0.3363132	test: 0.3352765	best: 0.3352765 (315)	total: 3s	remaining: 1m 13s
316:	learn: 0.3361618	test: 0.3351683	best: 0.3351683 (316)	total: 3.02s	remaining: 1m 13s
317:	learn: 0.3360085	test: 0.3350389	best: 0.3350389 (317)	total: 3.02s	remaining: 1m 13s
318:	learn: 0.3358519	test: 0.3348637	best: 0.3348637 (318)	total: 3.03s	remaining: 1m 13s
319:	learn: 0.3357269	test: 0.3347240	best: 0.3347240 (319)	total: 3.04s	remaining: 1m 13s
320:	learn: 0.3355951	test: 0.3345350	best: 0.3345350 (320)	total: 3.05s	remaining: 1m 12s
321:	learn: 0.3354647	test: 0.3343511	best: 0.3343511 (321)	total: 3.06s	remaining: 1m 13s
322:	learn: 0.3353495	test: 0.3341938	best: 0.3341938 (322)	total: 3.07s	remaining: 1m 12s
323:	learn: 0.3352743	test: 0.3340924	best: 0.3340924 (323)	total: 3.08s	remaining: 1m 12s
324:	learn: 0.3351032	test: 0.3339380	best: 0.3339380 (324)	total: 3.09s	remaining: 1m 12s
32

424:	learn: 0.3235264	test: 0.3241114	best: 0.3241114 (424)	total: 4s	remaining: 1m 11s
425:	learn: 0.3234646	test: 0.3240654	best: 0.3240654 (425)	total: 4.01s	remaining: 1m 11s
426:	learn: 0.3233441	test: 0.3240263	best: 0.3240263 (426)	total: 4.03s	remaining: 1m 11s
427:	learn: 0.3232764	test: 0.3239637	best: 0.3239637 (427)	total: 4.03s	remaining: 1m 11s
428:	learn: 0.3232024	test: 0.3239059	best: 0.3239059 (428)	total: 4.04s	remaining: 1m 11s
429:	learn: 0.3231243	test: 0.3238610	best: 0.3238610 (429)	total: 4.05s	remaining: 1m 11s
430:	learn: 0.3230405	test: 0.3237960	best: 0.3237960 (430)	total: 4.06s	remaining: 1m 11s
431:	learn: 0.3229014	test: 0.3237172	best: 0.3237172 (431)	total: 4.07s	remaining: 1m 11s
432:	learn: 0.3228147	test: 0.3237220	best: 0.3237172 (431)	total: 4.08s	remaining: 1m 11s
433:	learn: 0.3227325	test: 0.3237206	best: 0.3237172 (431)	total: 4.09s	remaining: 1m 11s
434:	learn: 0.3226478	test: 0.3236689	best: 0.3236689 (434)	total: 4.1s	remaining: 1m 11s
435

...

7601:	learn: 0.1199807	test: 0.2868179	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.26s
7602:	learn: 0.1199660	test: 0.2868131	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.25s
7603:	learn: 0.1199462	test: 0.2868118	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.24s
7604:	learn: 0.1199422	test: 0.2868105	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.23s
7605:	learn: 0.1199350	test: 0.2868087	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.22s
7606:	learn: 0.1199218	test: 0.2868051	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.21s
7607:	learn: 0.1199015	test: 0.2868098	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.2s
7608:	learn: 0.1198858	test: 0.2868034	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.19s
7609:	learn: 0.1198839	test: 0.2868023	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.18s
7610:	learn: 0.1198598	test: 0.2867905	best: 0.2867841 (7580)	total: 1m 21s	remaining: 4.17s
7611:	learn: 0.1198460	test: 0.2867891	best: 0.2867841 (7580)	total: 1m

7692:	learn: 0.1187153	test: 0.2867559	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.29s
7693:	learn: 0.1187033	test: 0.2867701	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.28s
7694:	learn: 0.1186821	test: 0.2867701	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.27s
7695:	learn: 0.1186758	test: 0.2867705	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.26s
7696:	learn: 0.1186635	test: 0.2867768	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.25s
7697:	learn: 0.1186623	test: 0.2867778	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.24s
7698:	learn: 0.1186616	test: 0.2867776	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.23s
7699:	learn: 0.1186401	test: 0.2867903	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.22s
7700:	learn: 0.1186294	test: 0.2867991	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.21s
7701:	learn: 0.1186173	test: 0.2867978	best: 0.2867559 (7692)	total: 1m 22s	remaining: 3.19s
7702:	learn: 0.1185949	test: 0.2867951	best: 0.2867559 (7692)	total: 1

7785:	learn: 0.1175283	test: 0.2867184	best: 0.2867184 (7785)	total: 1m 23s	remaining: 2.29s
7786:	learn: 0.1175124	test: 0.2867227	best: 0.2867184 (7785)	total: 1m 23s	remaining: 2.28s
7787:	learn: 0.1175016	test: 0.2867163	best: 0.2867163 (7787)	total: 1m 23s	remaining: 2.27s
7788:	learn: 0.1174864	test: 0.2867118	best: 0.2867118 (7788)	total: 1m 23s	remaining: 2.26s
7789:	learn: 0.1174698	test: 0.2867012	best: 0.2867012 (7789)	total: 1m 23s	remaining: 2.25s
7790:	learn: 0.1174567	test: 0.2866923	best: 0.2866923 (7790)	total: 1m 23s	remaining: 2.24s
7791:	learn: 0.1174430	test: 0.2866893	best: 0.2866893 (7791)	total: 1m 23s	remaining: 2.23s
7792:	learn: 0.1174273	test: 0.2866875	best: 0.2866875 (7792)	total: 1m 23s	remaining: 2.22s
7793:	learn: 0.1174177	test: 0.2866854	best: 0.2866854 (7793)	total: 1m 23s	remaining: 2.21s
7794:	learn: 0.1174031	test: 0.2866827	best: 0.2866827 (7794)	total: 1m 23s	remaining: 2.2s
7795:	learn: 0.1174019	test: 0.2866803	best: 0.2866803 (7795)	total: 1m

7879:	learn: 0.1164040	test: 0.2865568	best: 0.2865552 (7876)	total: 1m 24s	remaining: 1.29s
7880:	learn: 0.1163938	test: 0.2865522	best: 0.2865522 (7880)	total: 1m 24s	remaining: 1.28s
7881:	learn: 0.1163735	test: 0.2865522	best: 0.2865522 (7880)	total: 1m 24s	remaining: 1.26s
7882:	learn: 0.1163731	test: 0.2865504	best: 0.2865504 (7882)	total: 1m 24s	remaining: 1.25s
7883:	learn: 0.1163603	test: 0.2865566	best: 0.2865504 (7882)	total: 1m 24s	remaining: 1.24s
7884:	learn: 0.1163511	test: 0.2865513	best: 0.2865504 (7882)	total: 1m 24s	remaining: 1.23s
7885:	learn: 0.1163463	test: 0.2865498	best: 0.2865498 (7885)	total: 1m 24s	remaining: 1.22s
7886:	learn: 0.1163394	test: 0.2865499	best: 0.2865498 (7885)	total: 1m 24s	remaining: 1.21s
7887:	learn: 0.1163340	test: 0.2865493	best: 0.2865493 (7887)	total: 1m 24s	remaining: 1.2s
7888:	learn: 0.1163178	test: 0.2865500	best: 0.2865493 (7887)	total: 1m 24s	remaining: 1.19s
7889:	learn: 0.1162918	test: 0.2865534	best: 0.2865493 (7887)	total: 1m

7972:	learn: 0.1152010	test: 0.2864959	best: 0.2864959 (7972)	total: 1m 25s	remaining: 290ms
7973:	learn: 0.1151875	test: 0.2864949	best: 0.2864949 (7973)	total: 1m 25s	remaining: 279ms
7974:	learn: 0.1151748	test: 0.2864992	best: 0.2864949 (7973)	total: 1m 25s	remaining: 268ms
7975:	learn: 0.1151682	test: 0.2865083	best: 0.2864949 (7973)	total: 1m 25s	remaining: 257ms
7976:	learn: 0.1151521	test: 0.2865048	best: 0.2864949 (7973)	total: 1m 25s	remaining: 247ms
7977:	learn: 0.1151413	test: 0.2865082	best: 0.2864949 (7973)	total: 1m 25s	remaining: 236ms
7978:	learn: 0.1151265	test: 0.2865096	best: 0.2864949 (7973)	total: 1m 25s	remaining: 225ms
7979:	learn: 0.1151063	test: 0.2865116	best: 0.2864949 (7973)	total: 1m 25s	remaining: 215ms
7980:	learn: 0.1150905	test: 0.2865117	best: 0.2864949 (7973)	total: 1m 25s	remaining: 204ms
7981:	learn: 0.1150819	test: 0.2865162	best: 0.2864949 (7973)	total: 1m 25s	remaining: 193ms
7982:	learn: 0.1150698	test: 0.2865045	best: 0.2864949 (7973)	total: 1

In [26]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,P1_F_simple_price_median,5.719946
1,P1_F_square_True_median,4.992143
2,total_square,3.675631
3,P1_F_max_distance_2_min,3.108508
4,region,3.027314
5,floor,3.013268
6,P1_F_catering_points_general_median,2.928093
7,P1_F_square_False_median,2.799438
8,city,2.320670
9,realty_type,2.113212


In [27]:
price0_preds = cat_model.predict(X1)
metrics_stat(np.array(price0_preds) , np.array(Y1))

{'mape': 6096.618867401655,
 'mdape': 4642.578300280301,
 'rmse': 113026.75377625941,
 'r2': -47722457302.669655,
 'raif_metric': 9.899999999999999}

In [30]:
price0_preds = cat_model.predict(X_val, ntree_end=3000)
metrics_stat(np.array(Y_val), np.exp(np.array(price0_preds)) /1.15)

{'mape': 0.22665884824858182,
 'mdape': 0.19168577931748065,
 'rmse': 54626.78576140563,
 'r2': 0.5821038003078188,
 'raif_metric': 1.1517356498910296}

In [ ]:
{'mape': 0.26220391700430296,
 'mdape': 0.21319598144575228,
 'rmse': 48166.22978726944,
 'r2': 0.6751054446446725,
 'raif_metric': 1.3396794293308827}

In [72]:
X1.iloc[:,126:]

,realty_type,street,floor
0,110,S5845,-999
1,10,S27968,-999
2,10,S23597,-999
3,10,S10587,-999
4,10,S11220,-999
5,10,S6752,2.0
6,10,S6703,-999
7,10,S27593,-999
8,10,S19643,-999
9,10,S28820,-999


In [93]:

Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds3 = cat_model.predict(test_data)/1.15

In [96]:
price0_preds = (price0_preds1 + price0_preds2 + price0_preds3)/3

In [97]:
pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("ens_for food.csv", index=False)